In [17]:
!pip install ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [18]:
!pip install google-generativeai

In [19]:
# Import necessary libraries
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import widgets, Layout, Button, HTML, HBox, VBox, Text, IntSlider, Dropdown
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import display, clear_output
import textwrap
import google.generativeai as genai

In [20]:
gemini_api_key = '' ## insert api key
genai.configure(api_key=gemini_api_key)
model_name = "gemini-pro"
model = genai.GenerativeModel(model_name)

In [21]:
app_output = widgets.Output()
user_inputs = []

In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import textwrap
import re
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine



def main_chat_screen(model, additional_inf=[]):
    with app_output:
        model = model
        problem_input = widgets.Textarea(
                                         placeholder='Write your problem here...',
                                         layout=widgets.Layout(width='75%', height='100px')
                                         )

        submit_button = widgets.Button( description='Submit Problem',
                                            button_style='success',
                                            layout=widgets.Layout(width='200px', height='40px')
                                            )

        submit_button.style.button_color = '#d3d3d3'
        menu_button = create_menu()

        top_bar = widgets.HBox([menu_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
        main_area = widgets.VBox(
        [widgets.HTML(value="<h2>Describe Your Problem</h2>"), problem_input, submit_button],
        layout=widgets.Layout(align_items='center', width='100%')
        )
        # menu_button_box = widgets.HBox([menu_button], layout=widgets.Layout(justify_content='flex-start'))
        chat_vbox = widgets.VBox([top_bar, main_area])

        problem_refinement = ProblemRefinement(model)
        def on_submit_clicked(b):
            # Pass the current problem description and output area for processing

            problem_refinement.on_submit_clicked(problem_input.value.strip(), app_output)
        submit_button.on_click(on_submit_clicked)


        display(chat_vbox)


import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import textwrap
import re
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import json
import numpy as np

class ProblemRefinement:
    def __init__(self, model):  # Corrected __init__ method
        self.model = model
        self.problem_description='problem_description'
        self.mece_decomposition="mece"
        self.additional_notes='additional_notes'
        self.structured_execution_plan='structured_execution_plan'
        self.structured_clarification_questions='structured_clarification_questions'
        self.problem_rating = 1
        self.menu_button = create_menu()
        self.top_bar = widgets.HBox([self.menu_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))

    def get_mece_decomposition(self):
      return self.mece_decomposition


    def get_gemini_pro_embedding(self, text, task_type="SEMANTIC_SIMILARITY"):
        """
        Generate embeddings for given text using the genai embed_content method.

        Args:
            text (str): The text to embed.
            task_type (str): The type of task for which the embedding is generated. Defaults to SEMANTIC_SIMILARITY.

        Returns:
            A list representing the embedding vector for the input text.
        """
        model = "models/embedding-001"

        content = text
        title = "Document Title" if task_type == "RETRIEVAL_DOCUMENT" else None

        try:
            if title:
                result = genai.embed_content(model=model, content=content, task_type=task_type, title=title)
            else:
                result = genai.embed_content(model=model, content=content, task_type=task_type)
            return result['embedding']
        except AttributeError:
            return None

    def process_and_embed_categories(self, categories_dict):
        categories = [{"name": key, "description": value} for key, value in categories_dict.items()]
        for category in categories:
            # Attempt to embed description, handle potential missing method
            category['embedding'] = self.get_gemini_pro_embedding(category['description'], "semantic_similarity")
        #print("embedding:",categories)
        return categories

    def calculate_similarity(self, vec1, vec2):
        """Calculate the cosine similarity between two vectors using sklearn."""
        # cosine_similarity expects 2D arrays, so we need to reshape our 1D arrays
        vec1 = np.array(vec1).reshape(1, -1)
        vec2 = np.array(vec2).reshape(1, -1)
        return cosine_similarity(vec1, vec2)[0][0]


    def check_mece(self, categories):
        threshold = 0.5  # Example similarity threshold
        total_similarity = 0
        comparisons_made = 0
        overlapping_categories = []


        for i in range(len(categories)):
            for j in range(i + 1, len(categories)):
                # Check if embeddings exist for both categories before comparison
                if 'embedding' in categories[i] and 'embedding' in categories[j]:
                    similarity = self.calculate_similarity(categories[i]['embedding'], categories[j]['embedding'])
                    total_similarity += similarity
                    comparisons_made += 1

                    if similarity > threshold:
                        overlapping_categories.append((categories[i]['name'], categories[j]['name'], similarity))

        # Calculate the average similarity if at least one comparison was made
        if comparisons_made > 0:
            average_similarity = total_similarity / comparisons_made
        if average_similarity>0.6:
            return overlapping_categories
        else:
            return 'good'

    def on_submit_clicked(self, problem_description, output_area):
        with output_area:
            clear_output()
            html_widget = widgets.HTML(f"<h3>Loading...</h3>")
            container_vbox = widgets.VBox([self.top_bar, html_widget],layout=widgets.Layout(align_items='center'))
            display(container_vbox)
            if problem_description:
                problem_statement, mece_decomposition, additional_notes = self.decompose_problem_mece(problem_description)
                if not problem_statement or problem_statement.lower() == "not provided":
                    problem_statement = f"Problem submitted: {problem_description}"
                self.display_mece_html(problem_statement, mece_decomposition, additional_notes,output_area)


    def generate_mece_template(self,problem_statement):
            template = f"""
            Problem Statement:
            {problem_statement}
            problem_statement is:

            MECE Decomposition:
            Break down the problem into up to six categories following the Mutually Exclusive, Collectively Exhaustive (MECE) principle. For each category, give a short description that explains its relevance to the problem.

            1. Category 1: [Category Name]
               - Description: Explain how this category encompasses a specific aspect of the problem.

            2. Category 2: [Category Name]
               - Description: Outline the unique elements of the problem this category addresses.

            3. Category 3: [Category Name]
               - Description: Describe the role this category plays in the overall problem context.

            4. Category 4: [Category Name] (If applicable)
               - Description: Highlight the distinct issues or solutions this category covers.

            5. Category 5: [Category Name] (If applicable)
               - Description: Provide insights into how this category contributes to understanding or solving the problem.

            6. Category 6: [Category Name] (If applicable)
               - Description: Elucidate the specific aspects of the problem this category aims to address.

            Additional Notes:
            Include any other relevant information or assumptions that didn't fit into the categories but are important for a comprehensive understanding of the problem.
            """
            return template

    def parse_mece_to_dict(self, mece_text):
        """Parse MECE decomposition text into a dictionary of categories and descriptions."""
        categories_dict = {}
        try:
            current_category = ""
            for line in mece_text.split("\n"):
                line = line.strip()
                if "Category:" in line:
                    current_category = line.split("Category:")[1].strip()
                elif line.startswith("- Description:"):
                    description = line.split("- Description:", 1)[1].strip()
                    if current_category:
                        categories_dict[current_category] = description
        except Exception as e:
            print(f"Error parsing MECE text: {e}")
            return {}  # Return an empty dictionary in case of parsing failure

        return categories_dict


    def check_mece2(self, categories):
        threshold = 0.5  # Example similarity threshold
        total_similarity = 0
        num_comparisons = 0
        overlapping_categories = []

        for i in range(len(categories)):
            for j in range(i + 1, len(categories)):
                if 'embedding' in categories[i] and 'embedding' in categories[j]:
                    similarity = self.calculate_similarity(categories[i]['embedding'], categories[j]['embedding'])
                    total_similarity += similarity
                    num_comparisons += 1
                    if similarity > threshold:
                        overlapping_categories.append((categories[i]['name'], categories[j]['name'], similarity))

        average_similarity = total_similarity / num_comparisons if num_comparisons > 0 else 0
        return overlapping_categories, average_similarity

    def generate_refinement_prompt(self, problem_description, overlapping_categories):
        """
        Generate a prompt asking the model to refine the MECE decomposition based on identified overlaps.
        """
        overlap_description = "Identified overlaps: " + ", ".join(f"{cat1} and {cat2} (Similarity: {sim:.2f})" for cat1, cat2, sim in overlapping_categories)
        prompt = (f"Given the problem description: '{problem_description}', and the {overlap_description}, "
                  "please refine the MECE decomposition to address these overlaps. The goal is to modify or "
                  "reorganize the categories so that they adhere more closely to the Mutually Exclusive, Collectively "
                  "Exhaustive (MECE) principle.\n\n"
                  "Problem Statement:\n"
                  f"{problem_description}\n\n"
                  "MECE Decomposition:\n"
                  "Break down the problem into up to six categories following the MECE principle. For each category, "
                  "give a short description that explains its relevance to the problem.\n\n"
                  "1. Category 1: [Refined Category Name]\n"
                  "   - Description: [Refined Explanation]\n"
                  "2. Category 2: [Refined Category Name]\n"
                  "   - Description: [Refined Explanation]\n"
                  "(Continue this pattern for the remaining categories, addressing the identified overlaps and refining "
                  "the decomposition accordingly.)\n\n"
                  "Additional Notes:\n"
                  "Include any other relevant information or assumptions that didn't fit into the categories but are "
                  "important for a comprehensive understanding of the problem.")
        return prompt



    def refine_mece_decomposition(self, problem_description, initial_mece_decomposition,overlapping_categories, average_similarity):
        iteration_limit = 3  # Set a limit to avoid infinite loops
        average_similarity = 1  # Initialize to a value greater than 0.6 to start the loop
        mece_decomposition = initial_mece_decomposition  # Use the initial decomposition for the first iteration
        additional_notes = ""
        for iteration in range(iteration_limit):
            if average_similarity <= 0.6:
                # break
                pass

            if iteration >= 0:  # For subsequent iterations, generate new prompts based on feedback
                prompt = self.generate_refinement_prompt(problem_description, overlapping_categories)
                response = self.model.generate_content(prompt)
            prompt_categories = f"Based on the MECE decomposition {response.text}, create a dictionary where the keys are the MECE categories and the values are their descriptions. Each key should exclude the 'Category:' prefix for simplicity. Format the output as a dictionary."

            response_categories = self.model.generate_content(prompt_categories)
            categories_dict_text=response_categories.text
            prompt_mece = f"Given the dictionary {categories_dict_text}, convert it to a structured text format resembling this example: {mece_decomposition}"
            response_mece_decomposition = self.model.generate_content(prompt_mece)
            mece_decomposition_text=response_mece_decomposition.text
            categories_dict = self.parse_mece_to_dict(categories_dict_text)
            categories = self.process_and_embed_categories(categories_dict)
            overlapping_categories, average_similarity = self.check_mece2(categories)
            if average_similarity <= 0.6:
                mece_decomposition=mece_decomposition_text
        return mece_decomposition, additional_notes



    def decompose_problem_mece(self, problem_description):

        prompt = self.generate_mece_template(problem_description)
        response = self.model.generate_content(prompt)


        mece_decomposition_2 = """**1. Category 1: Desire**
        - Description: Reflects the underlying desire to consume pizza, encompassing the motivation and craving for the food.
        **2. Category 2: Availability**
        - Description: Addresses the accessibility of pizza, including factors such as availability at nearby restaurants, home delivery options, and potential time constraints.
        **3. Category 3: Resources**
        - Description: Considers the financial means and allocation of funds needed to purchase or prepare the pizza.
        **4. Category 4: Logistics**
        - Description: Outlines practicalities such as transportation to a restaurant, delivery arrangements, or cooking equipment if making pizza at home.
        **5. Category 5: Health Considerations**
        - Description: Takes into account dietary restrictions, allergies, and any potential health implications related to pizza consumption.
        **6. Category 6: Timing**
        - Description: Contemplates the ideal time to eat pizza, as it may be influenced by hunger levels, social plans, or other commitments."""

        try:
            problem_statement, mece_decomposition, additional_notes = self.extract_content_sections(response.text)
            is_empty = False

            # Check if any of the extracted sections are empty or None
            if not problem_statement or not mece_decomposition or not additional_notes:
                is_empty = True
            prompt_categories = f"Based on the MECE decomposition {response.text}, create a dictionary where the keys are the MECE categories and the values are their descriptions. Each key should exclude the 'Category:' prefix for simplicity. Format the output as a dictionary."

            response_categories = self.model.generate_content(prompt_categories)
            categories_dict_text=response_categories.text
            prompt_mece = f"Given the dictionary {categories_dict_text}, convert it to a structured text format resembling this example: {mece_decomposition_2}"
            response_mece_decomposition = self.model.generate_content(prompt_mece)
            mece_decomposition=response_mece_decomposition.text

        except Exception as e:
            print("[Error] Extraction Failed:", str(e))
        prompt_categories = f"Based on the MECE decomposition {mece_decomposition}, create a dictionary where the keys are the MECE categories and the values are their descriptions. Each key should exclude the 'Category:' prefix for simplicity. Format the output as a dictionary."

        response_categories = self.model.generate_content(prompt_categories)
        categories_dict_text=response_categories.text
        categories_dict=self.parse_mece_to_dict(categories_dict_text)
        categories = self.process_and_embed_categories(categories_dict)
        overlapping_categories, average_similarity=self.check_mece2(categories)
        if average_similarity>0.6:
        # # Now process the initial MECE decomposition to identify overlaps and refine if necessary
             mece_decomposition, additional_notes = self.refine_mece_decomposition(problem_description, mece_decomposition,overlapping_categories, average_similarity)
        return problem_statement, mece_decomposition, additional_notes


    def extract_content_sections(self, text):
        # Initialize variables for each section's content
        problem_statement = ""
        mece_decomposition = []
        additional_notes = []

        # Markers to identify the start of each section
        markers = {
            '*Problem Statement:*': 'problem_statement',
            '*MECE Decomposition:*': 'mece_decomposition',
            '*Additional Notes:*': 'additional_notes',
        }

        current_section = None

        for line in text.split('\n'):
            # Check if the line marks the beginning of a new section
            for marker, section in markers.items():
                if line.strip().startswith(marker):
                    current_section = section
                    break
            else:  # If the line doesn't match any marker, add it to the current section
                if current_section == 'problem_statement':
                    problem_statement += line.strip() + " "
                elif current_section == 'mece_decomposition':
                    if line.strip():  # Avoid adding empty lines
                        mece_decomposition.append(line.strip())
                elif current_section == 'additional_notes':
                    if line.strip():  # Avoid adding empty lines
                        additional_notes.append(line.strip())

        # Formatting the MECE Decomposition for display
        mece_formatted = "\n".join(mece_decomposition)
        additional_notes_formatted = "\n".join(additional_notes)


        return problem_statement.strip(), mece_formatted, additional_notes_formatted.strip()


    def handle_clarification(self, refined_description):
        problem_refinement = ProblemRefinement(model)
        clarification_answer_input = widgets.Textarea(
                                              placeholder='Type your clarification here...',
                                              layout=widgets.Layout(width='75%', height='100px')
                                          )
        html_widget = widgets.HTML(f"<h3>Please enter a clarification (Please be very spesific and provide as much details as possible)</h3></p>")
        clarification_submit_button = widgets.Button( description='Submit Problem',
                                                            button_style='success',
                                                            layout=widgets.Layout(width='200px', height='40px')
                                                            )

        clarification_submit_button.style.button_color = '#d3d3d3'

        def on_clarification_submitted(b):
            with app_output:
                clear_output(wait=True)
                user_clarification = clarification_answer_input.value
                problem_refinement.on_submit_clicked(user_clarification.strip(), app_output)

                self.handle_clarification(refined_description)
        clarification_submit_button.on_click(on_clarification_submitted)

        show_box = widgets.VBox([ self.top_bar,html_widget,clarification_answer_input, clarification_submit_button], layout=widgets.Layout(align_items='center'))
        display(show_box)


    def create_execution_plan_prompt(self, mece_decomposition, problem_description):
        """
        Constructs a prompt for the model based on the MECE decomposition to generate an execution plan.
        """
        prompt = (
        f"Given the MECE decomposition: {mece_decomposition} and my problem description: '{problem_description}', "
        "create an execution plan that addresses each category in the MECE decomposition. "
        "Please focus specifically on the categories provided and ensure the plan is comprehensive. "
        "Additionally, identify up to six relevant questions for clarification to ensure a comprehensive understanding. "
        "The questions should help in further refining the execution plan and be directly related to the categories outlined in the MECE decomposition."
        )
        return prompt

    def generate_content(self, prompt):
        return self.model.generate_content(prompt).text


    def extract_execution_plan(self, initial_execution_plan_text):
        prompt_execution_plan = f"""
            Given the details provided in the full execution plan below, please extract and structure the main points into categorized sections up to the 'Clarification Questions' section. Ignore any 'Clarification Questions' and focus solely on the execution plan categories and their details. Each category should be clearly titled and followed by bullet points listing the specific actions or considerations. Format your response in a structured manner similar to this example:

            '**Category**: [Category Title]
            * Action or consideration 1
            * Action or consideration 2'

            {initial_execution_plan_text.split('**Clarification Questions:**')[0]}  # This line splits the text and takes the part before 'Clarification Questions'
            """
        structured_execution_plan = self.generate_content(prompt_execution_plan)
        prompt_clarification_questions = f"""
          Based on the full execution plan provided, identify and list the relevant questions for clarification that are essential for refining the plan.
          Present the questions in a numbered format, focusing on aspects such as preferences, constraints, and specifics that need to be addressed.
          Example of the expected format:

          '1. What specific preferences are there for the project?
          2. Are there any constraints that need to be considered?'

          {initial_execution_plan_text}
          """
        structured_clarification_questions = self.generate_content(prompt_clarification_questions)
        return structured_execution_plan, structured_clarification_questions


    def display_mece_html(self, problem_statement, mece_decomposition, additional_notes, output_area):
        """
        Display the MECE decomposition and additional notes in the specified output area.
        """
        with app_output:
            clear_output(wait=True)
            self.mece_decomposition=mece_decomposition
            self.problem_description=problem_statement
            self.additional_notes=additional_notes
            html_content = f"<h3>Problem Statement:</h3><p>{problem_statement if problem_statement else 'Not provided'}</p>"
            if mece_decomposition.strip():
                html_content += f"<h3>MECE Decomposition:</h3><ul>"

                for line in mece_decomposition.split('\n'):
                    if "- " in line:
                        html_content += f"<li>{line}</li>"
                    else:

                        html_content += f"<li>{line}</li>"
                html_content += "</ul>"
            else:
                html_content += "<p>MECE Decomposition not detailed.</p>"

            if additional_notes.strip() and additional_notes.lower() != "none":
                html_content += f"<h3>Additional Notes:</h3><p>{additional_notes}</p>"
            else:
                html_content += "<p>No additional notes provided.</p>"
            feedback_options = ['Yes, proceed', 'No, I need to clarify further']
            html_content += f"<h3> Are you satisfied with the given categories? </h3>"
            feedback_inputs = widgets.RadioButtons(options=feedback_options)
            feedback_submit_button = widgets.Button(description="Submit Feedback")
            feedback_submit_button.style.button_color = '#d3d3d3'

            def on_feedback_submitted(b):
                with app_output:
                    clear_output(wait=False)
                    if feedback_inputs.value == 'Yes, proceed':
                        html_widget = widgets.HTML(f"<h3>Loading...</h3>")
                        container_vbox = widgets.VBox([self.top_bar, html_widget],layout=widgets.Layout(align_items='center'))
                        display(container_vbox)
                        prompt = self.create_execution_plan_prompt( self.mece_decomposition,self.problem_description)
                        initial_execution_plan = self.model.generate_content(prompt)
                        structured_execution_plan, structured_clarification_questions = self.extract_execution_plan(initial_execution_plan.text)
                        self.structured_execution_plan=structured_execution_plan
                        self.structured_clarification_questions=structured_clarification_questions
                        close = widgets.Button(description='X', button_style='info', layout=widgets.Layout(width='50px', height='30px'))
                        close.style.button_color = '#d3d3d3'
                        save_and_feedback_submit_button = widgets.Button(description="Save and submit feedback",layout=widgets.Layout(width='200px'))
                        save_and_feedback_submit_button.style.button_color = '#d3d3d3'
                        save_button = widgets.Button(description="Save")
                        button_box = widgets.HBox([save_and_feedback_submit_button, save_button], layout=Layout(justify_content='center'))
                        save_button.style.button_color = '#d3d3d3'
                        def on_close_clicked(b):
                            with app_output:
                                clear_output(wait = False)
                                main_chat_screen(model)

                        def on_save_button_clicked( b):
                            current_input = {
                                "problem_description": self.problem_description,
                                "mece_decomposition": self.mece_decomposition,
                                "additional_notes": self.additional_notes,
                                "structured_execution_plan": self.structured_execution_plan,
                                "structured_clarification_questions": self.structured_clarification_questions,
                                "problem_rating": self.problem_rating
                            }

                            user_inputs.append(current_input)

                            with app_output:
                                clear_output(wait=False)
                                print("Problem saved successfully!")
                                main_chat_screen(model)


                        def on_save_and_feedback_submit_button_clicked(b):
                            current_input = {
                                "problem_description": self.problem_description,
                                "mece_decomposition": self.mece_decomposition,
                                "additional_notes": self.additional_notes,
                                "structured_execution_plan": self.structured_execution_plan,
                                "structured_clarification_questions": self.structured_clarification_questions,
                                "problem_rating": self.problem_rating
                            }

                            user_inputs.append(current_input)
                            with app_output:
                                clear_output(wait = False)
                                feedback_answer_input = widgets.Textarea(
                                              placeholder='Type your feedback here...',
                                              layout=widgets.Layout(width='75%', height='100px')
                                          )
                                html_widget = widgets.HTML(f"<h3>Please write feedback on your interaction with the system</h3></p>")
                                feedback_submit_button = widgets.Button( description='Submit feedback',
                                                                                    button_style='success',
                                                                                    layout=widgets.Layout(width='200px', height='40px')
                                                                                    )

                                feedback_submit_button.style.button_color = '#d3d3d3'
                                def feedback_submit_button_clicked(b):
                                    with app_output:
                                        clear_output()
                                        main_chat_screen(model)
                                feedback_submit_button.on_click(feedback_submit_button_clicked)
                                show_box = widgets.VBox([ self.top_bar,html_widget,feedback_answer_input, feedback_submit_button], layout=widgets.Layout(align_items='center'))
                                display(show_box)


                        close.on_click(on_close_clicked)
                        save_button.on_click(on_save_button_clicked)
                        save_and_feedback_submit_button.on_click(on_save_and_feedback_submit_button_clicked)

                        html_content = "<h3>Problem Statement:</h3>"
                        html_content += f"<p>{problem_statement}</p>"

                        html_content += "<h3>Execution Plan:</h3>"
                        html_content += "<ul>"
                        for category in structured_execution_plan.split("**")[1:]:
                            if category.strip():
                                parts = category.split("*", 1)
                                if len(parts) == 2:
                                    title, items = parts
                                    html_content += f"<li><strong>{title.strip()}</strong><ul>"
                                    for item in items.split("*"):
                                        if item.strip():
                                            html_content += f"<li>{item.strip()}</li>"
                                    html_content += "</ul></li>"
                                else:
                                    html_content += f"<li><strong>{parts[0].strip()}</strong></li>"
                        html_content += "</ul>"

                        html_content += "<h3>Clarification Questions:</h3>"
                        html_content += "<ol>"

                        questions_list = re.split(r'\n\d+\.\s', structured_clarification_questions)
                        if questions_list and questions_list[0] == "**Relevant Questions for Clarification**":
                            questions_list[1] = questions_list[0] + questions_list[1]
                            questions_list.pop(0)

                        html_content += "<h3>Clarification Questions:</h3><ol>"
                        for question in questions_list:
                            if question.strip():
                                html_content += f"<li>{question.strip()}</li>"
                        html_content += "</ol>"

                        html_widget = widgets.HTML(value=html_content, layout=widgets.Layout( justify_content='center', align_items='center'))
                        container_box = widgets.VBox([close,html_widget, button_box], layout=widgets.Layout(justify_content='center'))
                        clear_output(wait = False)
                        display(container_box)

                    else:
                        self.handle_clarification(problem_statement)

            feedback_submit_button.on_click(on_feedback_submitted)
            feedback_box = widgets.VBox([feedback_inputs, feedback_submit_button], layout=Layout(justify_content='center'))
            html_widget = widgets.HTML(value=html_content)

            all_box = widgets.VBox([self.top_bar, html_widget, feedback_box], layout=Layout(justify_content='center'))
            display(all_box)



In [23]:
# with app_output:
#   clear_output()

In [24]:
# # log_in()
# main_chat_screen(model)
# display(app_output)
# #if username_input.value == 'testUsername' and password_input.value == '123456':

In [25]:
def create_menu():
    # Create a button for the menu
    menu_button = widgets.Button(description='☰', button_style='info', layout=widgets.Layout(width='50px', height='30px'))
    menu_button.style.button_color = '#d3d3d3'

    menu_options = widgets.Dropdown(
        options=[('Main page', 'main_page'), ('Take questionnaire', 'questionnaire'),
                ('History', 'history'), ('Log out', 'logout')],
        description='Menu:',
        disabled=False,
        layout={'visibility': 'hidden'}
    )
    menu_output = widgets.Output()

    # Function to handle menu actions
    def on_menu_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            option = change['new']
            with app_output:
                clear_output(wait=True)
                if option == 'questionnaire':
                    with app_output:
                        clear_output(wait=False)
                        questioneer_screen()
                elif option == 'main_page':
                    with app_output:
                        clear_output(wait=False)
                        main_chat_screen(model)
                elif option == 'history':
                    with app_output:
                        clear_output(wait=False)
                        history_screen()
                elif option == 'logout':
                    with app_output:
                        clear_output(wait=False)
                        log_in()
    menu_options.observe(on_menu_change, names='value')

    def on_menu_button_clicked(b):
        menu_options.layout.visibility = 'visible' if menu_options.layout.visibility == 'hidden' else 'hidden'

    menu_button.on_click(on_menu_button_clicked)
    menu = widgets.VBox([menu_button, menu_options])

    return menu

In [26]:
from ipywidgets import widgets, Layout, Button, HTML, HBox
from IPython.display import display, clear_output

# curr_problem = 0
# curr_problem_display = 1


def history_screen():
    global curr_problem
    global curr_problem_display
    curr_problem = 0
    curr_problem_display = 1

    with app_output:
        clear_output(wait=True)
        style = """
        <style>
            .custom-box {
                background: #EEEEEE;
                border-radius: 20px;
                border: 1px solid #CCCCCC;
                box-shadow: 5px 5px 20px #DDDDDD;
                padding: 20px;
                width: 90%;
                margin: 20px auto;
                position: relative;
            }
            .custom-header {
                color: #333333;
                margin-bottom: 20px;
                text-align: center;
            }
            .custom-text {
                color: #555555;
                text-align: center;
                margin-left: 20px;
            }
        </style>
        """

        close = widgets.Button(description='X', button_style='info', layout=widgets.Layout(width='50px', height='30px'))
        close.style.button_color = '#d3d3d3'

        # content_html = style  # Start with the style definitions

        prev_button = Button(description="Previous", layout=Layout(width='100px', height='30px'))
        next_button = Button(description="Next", layout=Layout(width='100px', height='30px'))
        prev_button.style.button_color = '#d3d3d3'
        next_button.style.button_color = '#d3d3d3'

        def show_problem(index, index_to_display):
            the_content_html = f"""
                <div class="custom-box">
                    <div class="custom-header">
                        <h3>Problem {index_to_display}</h3>
                    </div>
                    <h3>Problem {index_to_display}</h3>
                    <p><strong>Description:</strong> {user_inputs[index]['problem_description']}</p>
                    <p><strong>MECE Decomposition:</strong> {user_inputs[index]['mece_decomposition']}</p>
                    <p><strong>Additional Notes:</strong> {user_inputs[index]['additional_notes']}</p>
                    <p><strong>Execution Plan:</strong> {user_inputs[index]['structured_execution_plan']}</p>
                    <p><strong>Clarification Questions:</strong> {user_inputs[index]['structured_clarification_questions']}</p>
                    <p><strong>Rating:</strong> {user_inputs[index]['problem_rating']}</p>
                </div>
            """
            return the_content_html


        if len(user_inputs)>0:
            content_html = style + show_problem(curr_problem,curr_problem_display)
        else:
            content_html = style +f"""
                <div class="custom-box">
                    <div class="custom-header">
                        <h3>There is no problem in your history</h3>
                    </div>
                </div>
            """

        box_html = widgets.HTML(content_html,layout=widgets.Layout(align_items='center'))

        def on_close_clicked(b):
            with app_output:
                clear_output(wait=False)
                main_chat_screen(model)

        def on_next_button_clicked(b):
            global curr_problem
            global curr_problem_display
            with app_output:
                clear_output(wait=False)
                curr_problem += 1
                curr_problem_display += 1
                content_html = style + show_problem(curr_problem,curr_problem_display)
                what_to_display(content_html)


        def on_prev_button_clicked(b):
            global curr_problem
            global curr_problem_display
            # nonlocal curr_problem
            with app_output:
                clear_output(wait=False)
                curr_problem -= 1
                curr_problem_display -= 1
                content_html = style + show_problem(curr_problem,curr_problem_display)
                what_to_display(content_html)

        def what_to_display(content_html):
            # with app_output:
                # clear_output(wait=False)
                box_html = widgets.HTML(content_html,layout=widgets.Layout(align_items='center'))
                buttons = widgets.HBox([], layout=widgets.Layout(justify_content='center', width='100%'))
                if curr_problem<len(user_inputs)-1 and curr_problem > 0:
                    buttons = widgets.HBox([prev_button, next_button], layout=widgets.Layout(justify_content='center', width='100%'))
                elif curr_problem<len(user_inputs)-1 and curr_problem==0:
                    buttons = widgets.HBox([next_button], layout=widgets.Layout(justify_content='center', width='100%'))
                elif curr_problem==len(user_inputs)-1 and len(user_inputs) > 1:
                    buttons = widgets.HBox([prev_button], layout=widgets.Layout(justify_content='center', width='100%'))
                else:
                    buttons = widgets.HBox([], layout=widgets.Layout(justify_content='center', width='100%'))
                display_content = (widgets.VBox([close, box_html,buttons], layout=widgets.Layout(justify_content='center', width='100%')))
                display(display_content)


        close.on_click(on_close_clicked)
        prev_button.on_click(on_prev_button_clicked)
        next_button.on_click(on_next_button_clicked)
        what_to_display(content_html)


In [27]:
def questioneer_screen():
    with app_output:
        style = """
        <style>
            .custom-box {
                background: #EEEEEE;
                border-radius: 20px;
                border: 1px solid #CCCCCC;
                box-shadow: 5px 5px 20px #DDDDDD;
                padding: 20px;
                width: 600px;
                margin: 50px auto;
                position: relative;
            }
            .custom-header {
                color: #333333;
                margin-bottom: 20px;
            }
            .custom-text, .widget-label {
                color: #555555;
                text-align: left;
                margin-left: 20px;
            }
            .custom-input {
                width: 95%;
                margin: 10px 0px;
            }
            .button-style {
                margin: 10px;
                padding: 10px 20px;
                background-color: #90ee90;
                border: 1px solid #ddd;
                border-radius: 4px;
                font-size: 16px;
            }
        </style>
        """

        input_layout = Layout(width='95%')

        # Define input widgets with styles
        name_input = Text(description="Name:", placeholder="Your full name", layout=input_layout)
        age_input = Dropdown(description="Age:",options = range(5,100),placeholder="Your Age", layout=input_layout)
        country_input = Text(description="Country:", placeholder="Country of living",layout=input_layout)
        goal_input = Text(description="Main Goal:", placeholder="Your main goal with this app", layout=input_layout)

        box_html = widgets.HTML(f"""
            {style}
            <div class="custom-box">
                <div class="custom-header">
                    <h2>Quick Questionnaire</h2>
                </div>
                <div class="custom-text">
                    Please fill out this quick questionnaire to help us personalize your experience.
                </div>
            </div>
        """)



        inputs_vbox = VBox([name_input, age_input, country_input, goal_input],
                          layout=Layout(width='600px', margin='0 auto'))

        submit_button = Button(description="Submit", button_style='', layout=Layout(width='auto', margin='2px'))
        cancel_button = Button(description="Cancel", button_style='', layout=Layout(width='auto', margin='2px'))
        submit_button.style.button_color = '#d3d3d3' # or 'lightgreen' or 'rgb(144,238,144)'
        cancel_button.style.button_color = '#d3d3d3' # or 'lightgreen' or 'rgb(144,238,144)

        def on_submit_clicked(b):
            if not name_input.value.strip():
                print("Name is required.")
            elif not country_input.value:
                print("Country is required.")
            else:
                additional_inf = [name_input.value, age_input.value, country_input.value]
                with app_output:
                    clear_output(wait = False)
                    main_chat_screen(model, additional_inf)


        def on_cancel_clicked(b):
          with app_output:
              clear_output(wait = False)
              main_chat_screen(model)
            # Here you might want to redirect them back or clear the form

        submit_button.on_click(on_submit_clicked)
        cancel_button.on_click(on_cancel_clicked)

        # Group the buttons horizontally
        # buttons_vbox = VBox([cancel_button, submit_button], layout=Layout(width='100px',justify_content='center'))
        button_box = widgets.HBox([cancel_button, submit_button], layout=Layout(justify_content='center'))
        container = VBox([box_html, inputs_vbox, button_box])
        display(container)


In [28]:
from ipywidgets import widgets, Layout, Button, HTML, HBox
from IPython.display import display, clear_output

def explain_screen_next():

    with app_output:
        # Define the styles and layout for the message box
        style = """
        <style>
            .custom-box {
                background: #EEEEEE;
                border-radius: 20px;
                border: 1px solid #CCCCCC;
                box-shadow: 5px 5px 20px #DDDDDD;
                padding: 20px;
                width: 600px;
                height: 250px;
                margin: 50px auto;
                position: relative;
            }
            .custom-header {
                color: #333333;
                margin-bottom: 20px;
            }
            .custom-text {
                color: #555555;
                text-align: left;
                margin-left: 20px;
            }
        </style>
        """

        # Define the HTML for the box content
        box_html = HTML(f"""
            {style}
            <div class="custom-box">
                <div class="custom-header">
                    <h2>How to use this app?</h2>
                </div>
                <div class="custom-text">
                    <ul>
                        <li>Clearly define your problem and write it in the chat.</li>
                        <li>Get MECE categories for your problem and review them. </li>
                        <li>Get your MECE problem solving plan!</li>
                    </ul>
                </div>
            </div>
        """)

        display(box_html)
        button_layout = Layout(margin='2px', width='auto')

        questionnaire_button = Button(description="Quick questionnaire", button_style='', layout=button_layout)
        end_button = Button(description="End", button_style='', layout=button_layout)
        questionnaire_button.style.button_color = '#d3d3d3'
        end_button.style.button_color = '#d3d3d3'
        buttons = VBox([end_button,questionnaire_button ], layout=Layout(display='flex', flex_flow='row', align_items='center', justify_content='center', width='100%'))

        def on_questionnaire_clicked(b):
            with app_output:
                clear_output(wait = False)
                questioneer_screen()

        def on_end_clicked(b):
            with app_output:
              clear_output(wait = False)
              main_chat_screen(model)

        questionnaire_button.on_click(on_questionnaire_clicked)
        end_button.on_click(on_end_clicked)

        display(buttons)

In [29]:
from ipywidgets import widgets, Layout, Button, HTML, HBox
from IPython.display import display, clear_output


def explain_screen():
    with app_output:
        style = """
        <style>
            .custom-box {
                background: #EEEEEE;
                border-radius: 20px;
                border: 1px solid #CCCCCC;
                box-shadow: 5px 5px 20px #DDDDDD;
                padding: 20px;
                width: 600px;
                height: 250px;
                margin: 50px auto;
                position: relative;
            }
            .custom-header {
                color: #333333;
                margin-bottom: 20px;
            }
            .custom-text {
                color: #555555;
                text-align: left;
                margin-left: 20px;
            }
        </style>
        """


        box_html = HTML(f"""
            {style}
            <div class="custom-box">
                <div class="custom-header">
                    <h2>First Time Here?</h2>
                </div>
                <div class="custom-text">
                    <ul>
                        <li>Welcome to the MECE Problem-Solving App!</li>
                        <li>MECE stands for Mutually Exclusive, Collectively Exhaustive.</li>
                        <li>Let's break down your problem step by step for a clear solution.</li>
                    </ul>
                </div>
            </div>
        """)

        button_layout = Layout(margin='2px', width='auto')

        skip_button = Button(description="Skip", button_style='', layout=button_layout)
        next_button = Button(description="Next", button_style='', layout=button_layout)
        skip_button.style.button_color = '#d3d3d3'
        next_button.style.button_color = '#d3d3d3'

        def skip(b):
            with app_output:
                clear_output(wait=False)
                main_chat_screen(model)

        def next_screen(b):
            with app_output:
                clear_output(wait=False)
                explain_screen_next()

        skip_button.on_click(skip)
        next_button.on_click(next_screen)

        button_box = widgets.HBox([skip_button, next_button], layout=Layout(justify_content='center'))

        display(box_html, button_box)

In [30]:
from ipywidgets import widgets, Layout, VBox
from IPython.display import display, clear_output

def log_in():
    with app_output:
        clear_output(wait=True)
        username_input = widgets.Text(
            value='',
            placeholder='Enter Username',
            description='Username:',
            layout=Layout(width='300px')
        )

        password_input = widgets.Password(
            placeholder='Enter Password',
            description='Password:',
            layout=Layout(width='300px')
        )

        login_button = widgets.Button(
            description='Log In',
            button_style='success',
            layout=Layout(width='100px')
        )

        ui_output = widgets.Output()
        def show_next_screen():
            with app_output:
                clear_output()
                explain_screen()


        def on_login_clicked(b):
            if username_input.value == 'testUsername' and password_input.value == '123456':
                show_next_screen()
            else:
                with app_output:
                    clear_output()
                    display(widgets.HTML("<h2>Invalid username or password.</h2>"))
                    display(login_ui)

        login_button.on_click(on_login_clicked)
        login_ui = VBox([username_input, password_input, login_button], layout=Layout(display='flex', flex_flow='column', align_items='center', justify_content='center', width='100%'))
        display(login_ui)


In [31]:
with app_output:
  clear_output()

In [32]:
log_in()
display(app_output)

Output()